In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/ClimaMM/climamm.zip -d /content

unzip:  cannot find or open /content/drive/MyDrive/ClimaMM/climamm.zip, /content/drive/MyDrive/ClimaMM/climamm.zip.zip or /content/drive/MyDrive/ClimaMM/climamm.zip.ZIP.


In [ ]:
# !pip install transformers
# !pip install timm

In [ ]:
import glob
from itertools import chain
import os
import random
import zipfile

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, utils
from tqdm.notebook import tqdm

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

import seaborn as sns
import matplotlib.pyplot as plt

# import timm

from time import sleep
import warnings
from transformers import AdamW

from transformers import CLIPProcessor, CLIPModel

In [ ]:
BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-3

seed = 510
task= 'stance'
NUM_LABELS = 3

model_used = 'clip'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data, img_dir, split, transform=None):
        self.data = data[data['split'] == split].reset_index(drop=True)

        if task == 'target':
            self.data = self.data[self.data['hate'] == 1].reset_index(drop=True)

        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {}
        img_name = os.path.join(self.img_dir, self.data.loc[idx, 'name'])
        label = self.data.loc[idx, task]
        text = self.data.loc[idx, 'text']

        if self.transform:
          image = self.transform(image)

        try:
            sample["label"] = int(label)
            # sample["text"] = text
            sample["text"] = text[:77]
            sample["image"] = img_name
        except Exception as e:
            print(e)

        return sample

In [ ]:
csv_path = 'drive/MyDrive/ClimaMM/climamm.csv'
img_dir = '/content/climamm'

data = pd.read_csv(csv_path)

train_dataset = ImageDataset(data=data, img_dir=img_dir, split='train')
val_dataset = ImageDataset(data=data, img_dir=img_dir, split='val')
test_dataset = ImageDataset(data=data, img_dir=img_dir, split='test')

train_data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


dataloaders = {'Train': train_data_loader, 'Test': test_data_loader, 'Val': val_data_loader}


# for image, label in train_dataset:
#     pass

In [ ]:
print(len(train_data_loader))
print(len(val_data_loader))
print(len(test_data_loader))

In [ ]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()

        # self.fc1 = nn.Linear(1792, 128)

        #Image Only
        # self.fc1 = nn.Linear(1024, 128)
        self.fc1 = nn.Linear(768, 128)


        self.fc2 = nn.Linear(128, NUM_LABELS)

        self.dropout = nn.Dropout(0.25)

    def forward(self, input):

      linear_out = self.dropout(F.relu(self.fc1(input)))

      final_out = self.fc2(linear_out)

      return final_out

In [ ]:
device = 'cuda'
print(device)

clip = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)

model = Model().to(device)

In [ ]:
#optimizer
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps=1e-8)

processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

#Loss function
criterion = nn.CrossEntropyLoss()


In [ ]:
#training
def train():
  train_loss = []
  train_acc = []

  val_loss = []
  val_acc = []

  for epoch in range(0, EPOCHS):
    print('-'*50)
    print('Epoch {}/{}'.format(epoch+1, EPOCHS))

    for phase in ['Train', 'Val']:

      batch_loss = 0.0000   #live loss
      batch_acc = 0.0000   #live accuracy

      y_pred = []
      y_true = []

      if phase == 'Train':
          model.train()
      else:
          model.eval()

      with tqdm(dataloaders[phase], unit="batch", desc=phase) as tepoch:

        for idx, batch in enumerate(tepoch):
          labels = batch["label"].to(device)
          text = batch["text"]

          imgs = []

          img_paths = batch["image"]

          for path in img_paths:

            imgs.append((Image.open(path)))


          # Multimodal

          inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True).to(device)

          clip_output = clip(**inputs)

          img_embed = clip_output.vision_model_output.pooler_output

          text_embed = clip_output.text_model_output.pooler_output

          concat = torch.cat((text_embed, img_embed), 1)

          output = model(concat)

          # #Image only

          # inputs = processor(images=imgs, return_tensors="pt").to(device)

          # img_embed = clip.get_image_features(**inputs)

          # output = model(img_embed)

          # #Text only
          # output = model(text_embed)

          loss = criterion(output, labels)

          if phase == 'Train':

              #zero gradients
              optimizer.zero_grad()

              # Backward pass  (calculates the gradients)
              loss.backward()

              optimizer.step()             # Updates the weights

          batch_loss += loss.item()

          _, preds = output.data.max(1)

          y_pred.extend(preds.tolist())
          y_true.extend(labels.tolist())

          tepoch.set_postfix(loss = batch_loss/(idx+1))

          #accumulate loss and accuracy

        if phase == 'Train':
          train_loss.append(batch_loss/(idx+1))
          train_acc.append(batch_acc)

        else:
          val_loss.append(batch_loss/(idx+1))
          val_acc.append(batch_acc)

In [ ]:
!pip install torchmetrics

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import torchmetrics

def test():
  model.eval()
  y_pred = []
  y_true = []

  # auroc_list = []

  auroc = torchmetrics.AUROC(task='multiclass', num_classes = NUM_LABELS)

  with tqdm(dataloaders['Test'], unit="batch") as tepoch:
    for idx, batch in enumerate(tepoch):

      labels = batch["label"].to(device)
      text = batch["text"]

      imgs = []

      img_paths = batch["image"]

      for path in img_paths:

        imgs.append((Image.open(path)))

      # Multimodal
      inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True).to(device)

      clip_output = clip(**inputs)

      img_embed = clip_output.vision_model_output.pooler_output

      text_embed = clip_output.text_model_output.pooler_output

      concat = torch.cat((text_embed, img_embed), 1)

      output = model(concat)


      #  #Image only
      # inputs = processor(images=imgs, return_tensors="pt").to(device)

      # img_embed = clip.get_image_features(**inputs)

      # output = model(img_embed)


      #  #Text only
      #  output = model(text_embed)

      _, preds = output.data.max(1)

      preds_proxy = torch.sigmoid(output).detach().cpu()

      auroc.update(preds_proxy, batch["label"])

      y_pred.extend(preds.tolist())
      y_true.extend(labels.tolist())

  print("Model: ", model_used )
  print("Seed: ", seed )
  print("Task: ", task )

  print("Accuracy:{:.4f}".format(accuracy_score(y_true, y_pred) ))
  print("Recall:{:.4f}".format(recall_score(y_true, y_pred,average='macro') ))
  print("Precision:{:.4f}".format(precision_score(y_true, y_pred,average='macro') ))
  print("f1_score:{:.4f}".format(f1_score(y_true, y_pred,average='macro') ))

  print("AUC:{:.4f}".format(auroc.compute()))

  print(classification_report(y_true,y_pred))
  print(confusion_matrix(y_pred,y_true))


In [ ]:
train()
test()